# Multi-input LSTM

In [1]:
import random
import numpy as np
import torch
import torch.nn as nn

 
 
# define input sequence
in_seq1 = np.array([x for x in range(0,100,10)])
in_seq2 = np.array([x for x in range(5,105,10)])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])


# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))


# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))


# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    x,y = [],[]    
    for i in range(len(sequences)):       
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern       
        seq_x  = sequences[i:end_ix, :-1]
        seq_y  = sequences[end_ix -1, -1]
        
        x.append(seq_x)
        y.append(seq_y)
    
    return np.array(x),np.array(y).reshape(len(y),1)


from sklearn.preprocessing import MinMaxScaler

# 数据归一化处理函数
scaler = MinMaxScaler(feature_range=(-1, 1))

dataset = scaler.fit_transform(dataset)

n_timesteps = 3 # this is number of timesteps
    
# convert dataset into input/output
x, y = split_sequences(dataset, n_timesteps)

x = torch.tensor(x,dtype=torch.float32)
y = torch.tensor(y,dtype=torch.float32)

print(x.shape, y.shape)


torch.Size([8, 3, 2]) torch.Size([8, 1])


In [2]:
# LSTM 神经网络定义
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.randn(self.num_layers, x.size(0), self.hidden_dim)
        c0 = torch.randn(self.num_layers, x.size(0), self.hidden_dim)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        # print('out.shape',out.shape)
        out = self.fc(out[:, -1, :]) 
        return out


In [3]:
n_features = 2 # this is number of parallel inputs

mv_net = LSTM(input_dim = n_features, hidden_dim=32, output_dim=1, num_layers=1)
criterion = torch.nn.MSELoss(reduction='mean') # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-1)


In [4]:

train_episodes = 128
batch_size = 32

for t in range(train_episodes):
    for b in range(0,len(x),batch_size):
        
        x_batch = x[b:b+batch_size,:,:]        
        y_batch = y[b:b+batch_size]    

        output = mv_net(x_batch) 

        loss = criterion(output, y_batch)  
        
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()        
        
        
    if t % 10 == 0 :
        print('step : ' , t , 'loss : ' , loss.item())

step :  0 loss :  0.2860657870769501
step :  10 loss :  0.019263986498117447
step :  20 loss :  0.016997922211885452
step :  30 loss :  0.024637797847390175
step :  40 loss :  0.011430315673351288
step :  50 loss :  0.005103104282170534
step :  60 loss :  0.001886598183773458
step :  70 loss :  0.0016857306472957134
step :  80 loss :  0.002047975780442357
step :  90 loss :  0.0006495072739198804
step :  100 loss :  0.00048268146929331124
step :  110 loss :  0.0009519142913632095
step :  120 loss :  0.0002036448277067393


## Link
https://stackoverflow.com/questions/56858924/multivariate-input-lstm-in-pytorch  
https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/  
https://www.kaggle.com/code/jphoon/bitcoin-time-series-prediction-with-lstm  